In [9]:
import plotly
import plotly.express as px
import numpy as np

from class_definitions import *
import pandas as pd
from vivino_db import database
db = database.db
db.db_engine.table_names()

In [2]:
tables = {}
for table in db.db_engine.table_names():
    tables[table]=pd.read_sql_table(table,db.db_engine)

In [3]:
df = pd.read_csv('dataset.csv')

### Some updates and formatting that I forgot when compiling the data

In [4]:
region_country = pd.read_sql('select name as region,country_name from regions',db.db_engine)
df = pd.merge(df,region_country,on='region')

stmt="""
SELECT
    v.id,
    w.type_id
FROM
    vintages v
INNER JOIN
    wines w on v.wine_id = w.id
"""
types = pd.read_sql(stmt,db.db_engine)

df = pd.merge(df,types,on='id')

In [5]:
type_map = {
    1:'Red',
    2:'White',
    3:'Sparkling',
    4:'Rose'
}
df['type']=df['type_id'].map(type_map)

In [7]:
# messy manual updates for now
# need a better way to match vivino results with wineSearcher results
df.loc[31,'type_id']=3
df.loc[158,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Chateu Gassier', 'Cotes de Provence', 'Rare Rose Blend', 'Rose - Crisp and Dry', 'Tomato-based Dishes']
df.loc[53,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Weingut Karl Proidl',"Kremstal",'Riesling',"White - Green and Flinty", 'White Fish' ]
df.loc[4,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Weingut Koehler-Ruprecht','Pfalz','Pinot Noir','Red - Light and Perfumed','Meaty and Oily Fish']

### Type,Wine Style,Grape/Blend

In [29]:
i = []
for t in type_map.values():
    i.extend(df[(df['Wine Style'].isna()==False)&(df['Wine Style'].str.contains(t))&(df['type']!=t)].index)

temp = df.copy(deep=True) 
temp = df.drop(index=i)[df['Wine Style'].isna()==False]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [30]:
temp['Wine Style']=temp.apply(lambda x: x['Wine Style'].split('-')[-1].strip(),axis=1)

In [40]:
fig = px.sunburst(temp,path=['type','Wine Style','Grape/Blend'],
                  color='acidity',color_continuous_scale='PuBu',
                  color_continuous_midpoint=np.average(temp['acidity'])
                 )
fig.show(renderer='browser')

In [ ]:
fig = px.sunburst(temp,path=['type','Wine Style','Grape/Blend'],
                  color='body',color_continuous_scale='PuBu',
                  color_continuous_midpoint=np.average(temp['body'])
                 )
fig.show(renderer='browser')

### Incorporating Tasting Notes

In [47]:
tasting_notes=joblib.load('Wines/tasting_notes.pkl')

In [86]:
tasting_notes[0]

{4470482: [{'index': 0,
   'tag': 'blackcurrant',
   'note': '2 mentions of black fruit notes'},
  {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
  {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
  {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
  {'index': 3, 'tag': 'chard', 'note': '1 mentions of vegetal notes'},
  {'index': 3, 'tag': 'red beet', 'note': '1 mentions of vegetal notes'},
  {'index': 4, 'tag': 'chocolate', 'note': '1 mentions of oaky notes'}]}

In [76]:
records = []
for t in tasting_notes:
    d = {}
    for k,v in t.items():
        d['id']=k
        d['notes']=list({x['note'].split('mentions of')[-1].replace('') for x in v})
        d['tags']=list({x['tag'] for x in v})
    records.append(d)

In [105]:
records[0]

{'id': 4470482,
 'notes': ['2 mentions of black fruit notes',
  '1 mentions of dried fruit notes',
  '1 mentions of floral notes',
  '1 mentions of oaky notes',
  '1 mentions of vegetal notes'],
 'tags': ['red beet',
  'blackcurrant',
  'chard',
  'chocolate',
  'prune',
  'violet',
  'plum']}

In [77]:
for record in records[:5]:
    for note in record["notes"]:
        if 'black' in note:
            print(note.split(' ')[0])

2
6
1


In [57]:
notes_tags =[]
for t in tasting_notes:
    v=list(t.values())[0]
    notes_tags.extend(v)

In [66]:
notes = []
for nt in notes_tags[:5]:
    notes.append(nt['note'])

In [69]:
'black fruit' in notes[0]

True

In [79]:
unique_notes = set()
unique_tags = set()
for record in records:
    notes = [x.split(' mentions of ')[-1].replace(' notes','') for x in record['notes']]
    [unique_notes.add(x) for x in notes]
    [unique_tags.add(x) for x in record['tags']]

In [80]:
unique_notes

{'ageing',
 'black fruit',
 'citrus',
 'dried fruit',
 'earthy',
 'floral',
 'microbio',
 'oaky',
 'red fruit',
 'spices',
 'tree fruit',
 'tropical',
 'vegetal'}

In [104]:
tasting_notes[0]

{4470482: [{'index': 0,
   'tag': 'blackcurrant',
   'note': '2 mentions of black fruit notes'},
  {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
  {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
  {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
  {'index': 3, 'tag': 'chard', 'note': '1 mentions of vegetal notes'},
  {'index': 3, 'tag': 'red beet', 'note': '1 mentions of vegetal notes'},
  {'index': 4, 'tag': 'chocolate', 'note': '1 mentions of oaky notes'}]}

In [109]:
nt_map = []
for t in tasting_notes:
    d = {}
    for k,value in t.items():
        d = {}
        for v in value:
            d['note'] = v['note'].split('mentions of')[-1].replace(' notes','')
            d['tag']=v['tag']
            nt_map.append(d)

In [115]:
taste_df = pd.DataFrame.from_records(nt_map).drop_duplicates()

In [116]:
fig = px.sunburst(taste_df,path=['note','tag'])

In [117]:
fig.show(renderer='browser')